# Costa Rica Pipeline Notebook

This is the `Translation` part of the notebook where:
- Original dataset is imported from `Classification_Plots.zip`
- An English translated GeoDataFrame is created (`gdf_e`)
- Both SP and EN GeoDataFrame's are saved as `parquet` files in `"Costa_Rica_Data/Setup Output"`

## Section 1 - Setup

### Section 1.1 - Installing software and importing packages

In [32]:
!pip install pandas geopandas pyarrow


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
import geopandas as gpd
import pandas as pd

### Section 1.2 - Importing our Dataset

In [34]:
# We are importing the observational data from the ZIP file provided (which contains the shape file) into a GeoDataFrame
#gdf = gpd.read_file('../Costa_Rica_Data/Classification_Plots.zip')
gdf = gpd.read_parquet('../Costa_Rica_Data/Data Acquisition Output/extracted_gee_data/sp_basic_gee_data.parquet')

# These display information about the GeoDataFrame to confirm the contains are what we expected
display(gdf.crs)
display(gdf.columns)

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Index(['plotid', 'sampleid', 'Uso', 'Cobertura', 'Vegetacion', 'Herbaceas',
       'Pasto_Arb', 'Cultivo', 'Humedal', 'Terreno', 'Agua', 'Otra_clase',
       'SAF', 'Cambios15_', 'Gana_Perdi', 'geometry', 'BLUE', 'GREEN', 'NIR',
       'RED', 'SWIR1', 'SWIR2', 'altura2', 'aspect', 'aspectcos', 'aspectdeg',
       'aspectsin', 'brightness', 'clay_1mMed', 'diff', 'elevation', 'evi',
       'fpar', 'hand30_100', 'lai', 'mTPI', 'ndvi', 'ocs_1mMed', 'sand_1mMed',
       'savi', 'silt_1mMed', 'slope', 'topDiv', 'wetness'],
      dtype='object')

## Section 1.3 - English Translations

Renaming every column name to their respective English translation

In [35]:
# Insert code here for Section 1.3
k_clms = ['plotid','sampleid','Uso','Cobertura','Vegetacion','Herbaceas', 'Pasto_Arb', 'Cultivo','Humedal', 'Terreno','Agua','Otra_clase','SAF','Cambios15_','Gana_Perdi','geometry']
gdf_s=gdf[k_clms]
# gdf_s

# new column names
english_translations_predictors = {
    'Uso': 'Use',
    'Cobertura': 'CoverType',
    'Vegetacion': 'Vegetations',
    'Herbaceas': 'Herbaceous',
    'Pasto_Arb': 'GrasslandShrub',
    'Cultivo': 'CropsType',
    'Humedal': 'WetlandArea',
    'Terreno': 'LandType',
    'Agua': 'WaterBodyType',
    'Otra_clase': 'OtherClass',
    'SAF': 'SAF',
    'Cambios15_': 'Changes_15',
    'Gana_Perdi': 'Gain_Loss',
}

# Create a new GeoDataFrame with selected columns and translations
gdf_e = gdf[k_clms].copy()  # Creating a new GeoDataFrame
gdf_e.rename(columns=english_translations_predictors, inplace=True)  # Renaming columns

Renaming each predictor according to their respective English translation

In [36]:
# Translation dictionaries for each categorical predictor
translation_dicts = {
    'Use': { # num of missing values: 216
        'Bosque': 'Forest',
        'Pastos': 'Grasslands',
        'Humedal': 'Wetlands',
        'Otras clases': 'Other classes',
        'Agricultura': 'Agriculture',
        'Plantacion forestal': 'Forest plantation',
        'Sin informacion': 'No information',
        'None': 'None'
    },
    'CoverType': { # num of missing values: 216
        'Vegetacion': 'Vegetation',
        'Sin vegetacion': 'No vegetation',
        'Agua': 'Water',
        'Sin informacion': 'No information',
        'Nubes y sombras': 'Clouds and shadows',
        'None': 'None'
    },
     'Changes_15': {# num of missing values: 216
        'No se determina': 'Not determined',
        'No': 'No',
        'Si': 'Yes',
        'None': 'None'
    },
    'Vegetations': {
        'Arboles': 'Trees',
        'Herbaceas': 'Herbaceous plants',
        'None': 'None',
        'Palmas': 'Palms',
        'Arbustos': 'Shrubs',
        'Otra vegetacion': 'Other vegetation',
        'Saran': 'Saran (plastic cover)',
        'Plastico': 'Plastic'
    },
    'Herbaceous': {
        'None': 'None',
        'Gramineas': 'Grasses',
        'Otras Herbaceas': 'Other Herbaceous',
        'Musaceas': 'Plantains'
    },
    'GrasslandShrub': {
        'None': 'None',
        'Pastos mezclados (70-90%)': 'Mixed Pasture (70-90%)',
        'Pastos Puros (90-100%)': 'Pure Pasture (90-100%)',
        'Pastos Combinados (50-70%)': 'Combined Pasture (50-70%)'
    },
    'CropsType': { # 10 different crop types
        'None': 'None',
        'Pina': 'Pineapple',
        'Otro': 'Other',
        'Arroz': 'Rice',
        'Citricos': 'Citrus',
        'Cana': 'Sugarcane',
        'Palma': 'Palm',
        'Banano': 'Banana',
        'Melon': 'Melon',
        'Sandia': 'Watermelon',
        'Cafe': 'Coffee'
    },
     'WaterBodyType': { # 2 types of water bodies
        'None': 'None',
        'Continentales': 'Continental',
        'Mar�timas': 'Marine'
    },

    'WetlandArea': { # 5 wetland area types
        'None': 'None',
        'Pantano (Palustre)': 'Swamp (Marsh)',
        'Cuerpos de agua': 'Water bodies',
        'Yolillal': 'Yolillal_Plants',
        'Salinera': 'Salt marsh',
        'Manglar': 'Mangrove'
    },
    'LandType': {
        'None': 'None',
        'Otras superficies': 'Other surfaces', # forested areas?
        'Terreno descubierto': 'Exposed land', #  land degradation (soil is exposed)
        'Suelo desnudo': 'Bare Land' # lacks cover (deforestation, desert)
    },
    'OtherClass': {
        'None': 'None',
        'Edificado / Desarrollado': 'Built/Developed',
        'Suelo desnudo': 'Bare Land',
        'Nubes': 'Clouds',
        'Sombra de nubes': 'Cloud shadow',
        'Paramo': 'Páramo (high-altitude ecosystem)',
        'Playas y arenales': 'Beaches and sandbanks'
    },
    'SAF': {
        'None': 'None',
        'Cultivo Puro (90-100%)': 'Pure crop (90-100%)',
        'Cultivo mezclado (70-90%)': 'Mixed crop (70-90%)',
        'Cultivo Combinado (50-70%)': 'Combined crop (50-70%)'
    },
    'Gain_Loss': {
        'None': 'None',# third category no gain or loss
        'Perdida de Bosque': 'Forest loss',
        'Ganancia de Bosque': 'Forest gain'
    }
}

# Loop through each column and replace the Spanish values with the English translations
for col, trans_dict in translation_dicts.items():
    gdf_e[col] = gdf_e[col].replace(trans_dict)

Confirming the translations in the new Dataframe

In [37]:
# set display options to show more content
pd.set_option('display.max_colwidth', None)  # Allows full width of content in each column
pd.set_option('display.max_rows', None)      # Show all rows (if there are not too many unique values)

# Now, get and display unique values for each categorical feature
unique_values = gdf_e.select_dtypes(include=['object', 'category']).apply(lambda x: x.unique())

# Display the result
print(unique_values)

Use                             [Forest, Grasslands, Wetlands, Other classes, Agriculture, Forest plantation, No information, None]
CoverType                                              [Vegetation, No vegetation, Water, No information, Clouds and shadows, None]
Vegetations                       [Trees, Herbaceous plants, None, Palms, Shrubs, Other vegetation, Saran (plastic cover), Plastic]
Herbaceous                                                                             [None, Grasses, Other Herbaceous, Plantains]
GrasslandShrub                                    [None, Mixed Pasture (70-90%), Pure Pasture (90-100%), Combined Pasture (50-70%)]
CropsType                                [None, Pineapple, Other, Rice, Citrus, Sugarcane, Palm, Banana, Melon, Watermelon, Coffee]
WetlandArea                                              [None, Swamp (Marsh), Water bodies, Yolillal_Plants, Salt marsh, Mangrove]
LandType                                                                    

In [38]:
test_cols = ['BLUE', 'GREEN', 'NIR', 'RED', 'SWIR1', 'SWIR2', 'altura2', 'aspect', 'aspectcos', 'aspectdeg', 'aspectsin', 'brightness', 'clay_1mMed', 'diff', 'elevation', 'evi', 'fpar', 'hand30_100', 'lai', 'mTPI', 'ndvi', 'ocs_1mMed', 'sand_1mMed', 'savi', 'silt_1mMed', 'slope', 'topDiv', 'wetness']

gdf_e[test_cols] = gdf[test_cols]

display(gdf_e.columns)
display(gdf_e.shape)

Index(['plotid', 'sampleid', 'Use', 'CoverType', 'Vegetations', 'Herbaceous',
       'GrasslandShrub', 'CropsType', 'WetlandArea', 'LandType',
       'WaterBodyType', 'OtherClass', 'SAF', 'Changes_15', 'Gain_Loss',
       'geometry', 'BLUE', 'GREEN', 'NIR', 'RED', 'SWIR1', 'SWIR2', 'altura2',
       'aspect', 'aspectcos', 'aspectdeg', 'aspectsin', 'brightness',
       'clay_1mMed', 'diff', 'elevation', 'evi', 'fpar', 'hand30_100', 'lai',
       'mTPI', 'ndvi', 'ocs_1mMed', 'sand_1mMed', 'savi', 'silt_1mMed',
       'slope', 'topDiv', 'wetness'],
      dtype='object')

(101160, 44)

New translated gdf is named
∴ `gdf_e`

### Section 1.4 - Saving Data

In [39]:
# Saves the Spanish (original language) dataframe as a parquet file
#gdf_s.to_parquet("../Costa_Rica_Data/Setup Output/sp_setup_data.parquet", engine="pyarrow")

gdf_e.to_parquet("../Costa_Rica_Data/Data Acquisition Output/en_basic_gee_data.parquet", engine="pyarrow")

# Saves the English (translated language) dataframe as a parquet file
#gdf_e.to_parquet("../Costa_Rica_Data/Setup Output/en_setup_data.parquet", engine="pyarrow")